In [1]:
import polars as pl
import polars_ds as pds
import numpy as np

# This notebook illustrates the basic usage of this package

You need to create an environment with this package installed to run this notebook. (usually latest version)

# Num Extensions

In [2]:
size = 10_000
df = pl.DataFrame({
    "f": np.sin(list(range(size)))
    , "time_idx": range(size)
    , "dummy": ["a"] * (size // 2) + ["b"] * (size // 2)
    , "actual": np.round(np.random.random(size=size)).astype(np.int32)
    , "predicted": np.random.random(size=size)
    , "dummy_groups":["a"] * (size//2) + ["b"] * (size//2) 
}).with_columns(
    pds.random(0., 1.).alias("x1")
    , pds.random(0., 1.).alias("x2")
    , pds.random(0., 1.).alias("x3")
    , pds.random(0., 1.).alias("a")
    , pds.random(0., 1.).alias("b")
).with_columns(
    y = pl.col("x1") * 0.15 + pl.col("x2") * 0.3 - pl.col("x3") * 1.5 + pds.random() * 0.0001,
    y2 = pl.col("x1") * 0.13 + pl.col("x2") * 0.45 - pl.col("x3") * 0.1 + pds.random() * 0.0001
)
df.head()

f,time_idx,dummy,actual,predicted,dummy_groups,x1,x2,x3,a,b,y,y2
f64,i64,str,i32,f64,str,f64,f64,f64,f64,f64,f64,f64
0.0,0,"""a""",0,0.375402,"""a""",0.942222,0.439831,0.971564,0.161725,0.081282,-1.184052,0.223283
0.841471,1,"""a""",0,0.459718,"""a""",0.3503,0.32613,0.301961,0.925339,0.63425,-0.302463,0.162196
0.909297,2,"""a""",1,0.852819,"""a""",0.159627,0.749902,0.027656,0.485268,0.129957,0.207438,0.355489
0.14112,3,"""a""",1,0.19791,"""a""",0.610992,0.830041,0.19653,0.462807,0.884362,0.04594,0.433295
-0.756802,4,"""a""",1,0.22126,"""a""",0.867392,0.174458,0.243151,0.421551,0.758321,-0.182225,0.167


In [3]:
# Column-wise Jaccard Similarity. Result should be 0 as they are distinct
df.select(
    pds.jaccard_col("x1", pl.col("x2"))
)

x1
f64
0.0


In [4]:
# FFT. First is real part, second is complex part
# By default, this behaves the same as np's rfft, which returns a non-redundant 
# compact representation of fft output.
df.select(
    pds.rfft("f")
).head()

f
"array[f64, 2]"
"[1.939505, 0.0]"
"[1.939506, 0.000209]"
"[1.939508, 0.000418]"
"[1.939512, 0.000627]"
"[1.939518, 0.000835]"


In [5]:
# FFT. But return the full length
df.select(
    pds.rfft("f", return_full=True)
).shape

(10000, 1)

In [6]:
# Multiple Convolutions at once
# Modes: `same`, `left` (left-aligned same), `right` (right-aligned same), `valid` or `full`
# Method: `fft`, `direct`
# Currently slower than SciPy but provides parallelism because of Polars
df.select(
    pds.convolve("f", [-1, 0, 0, 0, 1], mode = "full", method = "fft"), # column f with the kernel given here
    pds.convolve("a", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
    pds.convolve("b", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
).head()

f,a,b
f64,f64,f64
1.3944e-15,-0.161725,-0.081282
-0.841471,-0.925339,-0.63425
-0.909297,-0.485268,-0.129957
-0.14112,-0.462807,-0.884362
0.756802,-0.259826,-0.677039


In [7]:
# Linear Regression
df.select(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

lstsq_coeffs
list[f64]
"[-0.492448, -0.345018]"


In [8]:
# Linear Regression, multi-target
df.select(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = [pl.col("y"), pl.col("y2")],
        add_bias=False
    )
).unnest("lstsq_coeffs")

y,y2
list[f64],list[f64]
"[-0.492448, -0.345018]","[0.087209, 0.40704]"


In [9]:
df.select(
    pds.lin_reg_report(
        # formulaic input is also available for lstsq related queries, 
        # or you can always use polars expressions, e.g. pl.col('x1') + 1, pl.col('x2').exp(), pl.col('x3').sin()
        "ln(x1+1)", "exp(x2)", "sin(x3)",
        target = "y",
        add_bias = True
    ).alias("report")
).unnest("report")

features,beta,std_err,t,p>|t|,0.025,0.975
str,f64,f64,f64,f64,f64,f64
"""ln(x1+1)""",0.218917,0.001663,131.635063,0.0,0.215657,0.222177
"""exp(x2)""",0.173932,0.000667,260.7488,0.0,0.172624,0.175239
"""sin(x3)""",-1.742118,0.001336,-1304.216985,0.0,-1.744737,-1.7395
"""__bias__""",-0.106742,0.001485,-71.895288,0.0,-0.109652,-0.103832


In [10]:
df.select(
    "dummy",
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    ).over(pl.col("dummy"))
)

dummy,lstsq_coeffs
str,list[f64]
"""a""","[-0.486552, -0.344752]"
"""a""","[-0.486552, -0.344752]"
"""a""","[-0.486552, -0.344752]"
"""a""","[-0.486552, -0.344752]"
"""a""","[-0.486552, -0.344752]"
…,…
"""b""","[-0.49842, -0.345308]"
"""b""","[-0.49842, -0.345308]"
"""b""","[-0.49842, -0.345308]"


In [11]:
# If you want prediction and residue instead of coefficients
df.select(
    "x1",
    "x2",
    "y",
    pds.lin_reg(
        "x1", pl.col("x2"),
        target = "y",
        add_bias=False, 
        return_pred=True
    ).alias("prediction")
).unnest("prediction").head()

x1,x2,y,pred,resid
f64,f64,f64,f64,f64
0.942222,0.439831,-1.184052,-0.615745,-0.568307
0.3503,0.32613,-0.302463,-0.285026,-0.017438
0.159627,0.749902,0.207438,-0.337337,0.544775
0.610992,0.830041,0.04594,-0.587261,0.633201
0.867392,0.174458,-0.182225,-0.487336,0.305112


In [12]:
df.group_by("dummy").agg(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

dummy,lstsq_coeffs
str,list[f64]
"""b""","[-0.49842, -0.345308]"
"""a""","[-0.486552, -0.344752]"


In [13]:
# Lasso
df.group_by("dummy").agg(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        l1_reg = 0.1,
        add_bias=False
    )
)

dummy,lstsq_coeffs
str,list[f64]
"""a""","[-0.312191, -0.177624]"
"""b""","[-0.320968, -0.17587]"


In [14]:
# R2 metric of lasso regressions on each group
df.group_by("dummy").agg(
    pds.query_r2(
        actual = pl.col("y"),
        pred = pds.lin_reg(
            pl.col("x1"), pl.col("x2"),
            target = pl.col("y"),
            l1_reg = 0.1,
            return_pred = True,
            add_bias=False
        ).struct.field("pred")
    ).alias("lasso_r2")
)

dummy,lasso_r2
str,f64
"""a""",-0.529623
"""b""",-0.568124


In [15]:
# Rolling regression
df.select(
    "y",
    "x1",
    "x2",
    pds.rolling_lin_reg(
        "x1", "x2",
        target = "y",
        window_size = 5,
        null_policy = "zero"
    ).alias("result")
).unnest("result")

y,x1,x2,coeffs,pred
f64,f64,f64,list[f64],f64
-1.184052,0.942222,0.439831,null,null
-0.302463,0.3503,0.32613,null,null
0.207438,0.159627,0.749902,null,null
0.04594,0.610992,0.830041,null,null
-0.182225,0.867392,0.174458,"[-0.869293, 0.429004]",-0.679175
…,…,…,…,…
-0.942163,0.835742,0.644598,"[-0.219512, -0.451975]",-0.474798
-0.672708,0.606915,0.550782,"[-0.32353, -0.396994]",-0.415012
-1.196937,0.573096,0.602933,"[0.341433, -1.250515]",-0.558303


In [16]:
# Conditional Entropy, should be 0 because x1 is an ID
df.select(
    pds.query_cond_entropy("y", "x1")
)

y
f64
-0.0


In [17]:
# Only want singular values (principal values?)
df.select(
    pds.singular_values("a", "b", "x1")
)

a
list[f64]
"[29.105964, 28.759142, 28.501461]"


In [18]:
# Singular values + The principal components
df.select(
    pds.pca("a", "b")
).unnest("a")

singular_value,weight_vector
f64,list[f64]
28.955855,"[-0.638535, 0.769593]"
28.656947,"[0.769593, 0.638535]"


In [19]:
# PC1
df.select(
    pds.principal_components("a", "b", k =1).alias("principal_components")
).unnest("principal_components").head()

pc1
f64
-0.107721
-0.169755
-0.276855
0.318073
0.247415


# ML Metrics

In [20]:
df.group_by("dummy_groups").agg(
    pds.query_l2("actual", "predicted").alias("l2"),
    pds.query_log_loss("actual", "predicted").alias("log loss"),
    pds.query_binary_metrics(actual="actual", pred="predicted").alias("combo")
).unnest("combo")


dummy_groups,l2,log loss,precision,recall,f,average_precision,roc_auc
str,f64,f64,f64,f64,f64,f64,f64
"""b""",0.329164,0.993235,0.497233,0.511798,0.504411,0.501376,0.511545
"""a""",0.338169,1.014774,0.490172,0.493339,0.491751,0.487677,0.491598


# Str Extension

In [21]:
size = 100_000
df2 = pl.DataFrame({
    "sen":["Hello, world! I'm going to church."] * size,
    "word":["words", "word"] * (size //2)
})
df2.head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""


In [22]:
# Tokenize
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase()).explode().unique()
)

sen
str
"""hello"""
"""going"""
"""church"""
"""world"""
"""to"""


In [23]:
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase(), stem=True).explode().unique()
)

sen
str
"""go"""
"""church"""
"""world"""
""""""
"""hello"""


In [24]:
df2.select(
    pds.str_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [25]:
# Damerau-Levenshtein
df2.select(
    pds.str_d_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [26]:
df2.select( # column "word" vs. the word "world"
    pds.str_leven("word", pl.lit("world"), return_sim = True)
).head()

word
f64
0.6
0.8
0.6
0.8
0.6


In [27]:
df2.filter(
    # This is way faster than computing ditance and then doing a filter
    pds.filter_by_levenshtein(pl.col("word"), pl.lit("world"), 1) # <= 1. 
).head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""


In [28]:
df = pl.DataFrame({
    "word":["apple", "banana", "pineapple", "asasasas", "sasasass"],
    "other_data": [1,2,3,4,5]
})
gibberish = ["asasasa", "sasaaasss", "asdasadadfa"]

In [29]:
df.select(
    # Nearest string
    pds.str_nearest("word", word = "banana")
)

word
str
"""banana"""


In [30]:
df.filter(
    # Filters to words that are similar to any word in vocab
    pds.similar_to_vocab(
        pl.col("word"),
        vocab = gibberish,
        threshold = 0.5,
        metric = "lv", # Levenshtein similarity. Other options: dleven, osa, jw
        strategy = "any" # True if the word is similar to any word in vocab. Other options: "all", "avg"
    )
)

word,other_data
str,i64
"""asasasas""",4
"""sasasass""",5


In [31]:
df.select(
    pds.str_leven("word", pl.lit("asasasa"), return_sim=True).alias("asasasa"),
    pds.str_leven("word", pl.lit("sasaaasss"), return_sim=True).alias("sasaaasss"),
    pds.str_leven("word", pl.lit("asdasadadfa"), return_sim=True).alias("asdasadadfa"),
    pds.str_fuzz("word", pl.lit("apples")).alias("LCS based Fuzz match - apples"),
    pds.str_osa("word", pl.lit("apples"), return_sim=True).alias("Optimal String Alignment - apples"),
    pds.str_jw("word", pl.lit("apples")).alias("Jaro-Winkler - apples"),
)


asasasa,sasaaasss,asdasadadfa,LCS based Fuzz match - apples,Optimal String Alignment - apples,Jaro-Winkler - apples
f64,f64,f64,f64,f64,f64
0.142857,0.111111,0.090909,0.833333,0.833333,0.966667
0.428571,0.333333,0.272727,0.166667,0.0,0.444444
0.111111,0.111111,0.090909,0.555556,0.444444,0.5
0.875,0.666667,0.545455,0.25,0.25,0.527778
0.75,0.777778,0.454545,0.25,0.25,0.527778


# Stats Extension

In [32]:
import numpy as np

df = pl.DataFrame({
    "a": [None, None] + list(np.random.normal(size = 998))
})
df.head()

a
f64
null
null
-0.262887
-1.102007
0.040405


In [33]:
# Genenrate random numbers, respecting null positions in reference column (pl.col("a"))
df.with_columns(
    pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal")
    ).alias("random_normal_that_respects_null_of_a")
).head()

a,random_normal,random_normal_that_respects_null_of_a
f64,f64,f64
null,1.708497,null
null,2.466918,null
-0.262887,0.129385,-0.548552
-1.102007,0.040371,1.405887
0.040405,1.585635,-0.432342


In [34]:
# Genenrate random string
df.with_columns(
    pds.random_str(min_size = 1, max_size = 5).alias("random_str"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 1, max_size = 5)
    ).alias("random_str_that_respects_null_of_a")
).head()

a,random_str,random_str_that_respects_null_of_a
f64,str,str
null,"""f4pD""",null
null,"""e1""",null
-0.262887,"""R""","""PI"""
-1.102007,"""iaRu""","""S9Vk"""
0.040405,"""db9""","""JWIz"""


In [35]:
# Genenrate fixed size random string, while respecting column a's nulls
df.with_columns(
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 5, max_size = 5)
    ).alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
-0.262887,"""ACCgS"""
-1.102007,"""lLfnS"""
0.040405,"""QkXLj"""


In [36]:
df.with_columns(
    # Sample from a normal distribution, using reference column "a" 's mean and std
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("test1") 
    # Sample from uniform distribution, with low = 0 and high = "a"'s max, and respect the nulls in "a"
    , pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random(lower = 0., upper = pl.col("a").max()).alias("test2")
    )
).with_columns(
    # Add a random pertubation to test1
    pds.perturb("test1", epsilon=0.001).alias("test1_perturbed")
).head()

a,test1,literal,test1_perturbed
f64,f64,f64,f64
null,-0.783559,null,-0.783261
null,0.353855,null,0.354165
-0.262887,-0.011766,1.897042,-0.011788
-1.102007,-0.792581,2.124058,-0.79242
0.040405,1.215775,1.669608,1.21567


In [37]:
# New in v0.3.5
# This way, we don't have a reference column, so we cannot respect nulls, but is more convenient to use.
df.with_columns(
    pds.random().alias("[0, 1)"),
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("Normal"),
    pds.random_int(0, 10).alias("Int from [0, 10)"),
).head()

a,"[0, 1)",Normal,"Int from [0, 10)"
f64,f64,f64,i32
null,0.027235,0.088935,1
null,0.942403,0.790212,6
-0.262887,0.521936,2.001769,7
-1.102007,0.005144,0.669645,3
0.040405,0.895641,0.227451,3


In [38]:
# Genenrate 2 random sample, both normally distributed
# Run Welch's t test on them, p value should be big since they have equal mean
# Run a normality test. Again, p value should be big since they are normally distributed 

df.with_columns(
    pds.random_normal(0.5, 1.0).alias("test1"),
    pds.random_normal(0.5, 2.0).alias("test2"),
).select(
    pds.ttest_ind("test1", "test2", equal_var=False).alias("t-test"),
    pds.normal_test("test1").alias("normality_test")
).select(
    pl.col("t-test").struct.field("statistic").alias("t-tests: statistics")
    , pl.col("t-test").struct.field("pvalue").alias("t-tests: pvalue")
    , pl.col("normality_test").struct.field("statistic").alias("normality_test: statistics")
    , pl.col("normality_test").struct.field("pvalue").alias("normality_test: pvalue")
)

t-tests: statistics,t-tests: pvalue,normality_test: statistics,normality_test: pvalue
f64,f64,f64,f64
0.143723,0.885739,5.490021,0.064248


In [39]:
size = 5_000
df = pl.DataFrame({
    "market_id": range(size),
}).with_columns(
    pl.col("market_id").mod(3),
    var1 = pds.random(),
    var2 = pds.random(),
    category_1 = pds.random_int(0, 5),
    category_2 = pds.random_int(0, 10),
)

df.head(5)

market_id,var1,var2,category_1,category_2
i64,f64,f64,i32,i32
0,0.294425,0.827601,1,8
1,0.670674,0.567675,3,6
2,0.251429,0.570609,2,1
0,0.996076,0.405968,0,3
1,0.3855,0.734565,3,9


In [40]:
# In dataframe statistical tests!
df.select(
    pds.ttest_ind("var1", "var2", equal_var=True).alias("t-test"),
    pds.chi2("category_1", "category_2").alias("chi2-test"),
    pds.f_test("var1", group = "category_1").alias("f-test")
)

t-test,chi2-test,f-test
struct[2],struct[2],struct[2]
"{-0.971911,0.331118}","{25.746132,0.897306}","{0.238624,0.916601}"


In [41]:
# Can also be done in group by context
print(
    df.group_by("market_id").agg(
        pds.ttest_ind("var1", "var2", equal_var=False).alias("t-test"),
        pds.chi2("category_1", "category_2").alias("chi2-test"),
        pds.f_test("var1", group = "category_1").alias("f-test")
    )
)

shape: (3, 4)
┌───────────┬──────────────────────┬──────────────────────┬─────────────────────┐
│ market_id ┆ t-test               ┆ chi2-test            ┆ f-test              │
│ ---       ┆ ---                  ┆ ---                  ┆ ---                 │
│ i64       ┆ struct[2]            ┆ struct[2]            ┆ struct[2]           │
╞═══════════╪══════════════════════╪══════════════════════╪═════════════════════╡
│ 0         ┆ {-0.604632,0.545465} ┆ {27.950572,0.828958} ┆ {0.064619,0.992327} │
│ 1         ┆ {-0.756586,0.449351} ┆ {37.365459,0.40619}  ┆ {0.191293,0.943018} │
│ 2         ┆ {-0.321324,0.747985} ┆ {36.327492,0.453395} ┆ {0.534485,0.710422} │
└───────────┴──────────────────────┴──────────────────────┴─────────────────────┘


In [42]:
# Benford's law
df.select(
    first_digit_cnt = pds.query_first_digit_cnt(pl.col("var1")).explode()
).with_columns(
    # This doesn't follow benford's law because it is random data
    first_digit_distribution = pl.col("first_digit_cnt") / pl.col("first_digit_cnt").sum()
)

first_digit_cnt,first_digit_distribution
u32,f64
538,0.1076
556,0.1112
533,0.1066
577,0.1154
557,0.1114
576,0.1152
549,0.1098
553,0.1106
561,0.1122


# Nearest Neighbors Related Tasks

These queries can be very slow when data/dimension gets huge, even when processed in parallel.

In [43]:
import polars_ds as pds
size = 2000
df = pl.DataFrame({
    "id": range(size), 
}).with_columns(
    pds.random().alias("var1"),
    pds.random().alias("var2"),
    pds.random().alias("var3"),
    pds.random().alias("r"),
    (pds.random() * 10).alias("rh"),
    pl.col("id").cast(pl.UInt32)
)

In [44]:
# Get neighbor count. The point itself is always considered a neighbor to itself.
df.with_columns(
    pds.query_nb_cnt(
        pl.col("var1"), "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        r = 0.1, # radius 
        dist = "inf", # L Infinity distance 
        parallel = True 
    ).alias("nb_l_inf_cnt")
).head() 

id,var1,var2,var3,r,rh,nb_l_inf_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.909291,0.883797,0.050987,0.041203,9.03286,17
1,0.28296,0.561952,0.20464,0.91307,6.259133,16
2,0.202197,0.700336,0.297267,0.436134,3.891523,14
3,0.517782,0.076448,0.910966,0.864982,4.12844,17
4,0.208706,0.480551,0.367441,0.701084,3.39438,18


In [45]:
df.with_columns(
    pds.query_nb_cnt(
        "var1", "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        r = pl.col("r"), # radius be an expression too
        dist = "l1", # L 1 distance 
        parallel = True 
    ).alias("nb_l1_r_cnt")
).head()

id,var1,var2,var3,r,rh,nb_l1_r_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.909291,0.883797,0.050987,0.041203,9.03286,1
1,0.28296,0.561952,0.20464,0.91307,6.259133,1076
2,0.202197,0.700336,0.297267,0.436134,3.891523,190
3,0.517782,0.076448,0.910966,0.864982,4.12844,624
4,0.208706,0.480551,0.367441,0.701084,3.39438,709


In [46]:
# Get ids of the k nearest neighbors. 
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = "id",  # pl.col("id"), str | pl.Expr
        k = 3, 
        dist = "l2", # squared l2
        parallel = True
    ).alias("best friends")
).head() 

id,var1,var2,var3,r,rh,best friends
u32,f64,f64,f64,f64,f64,list[u32]
0,0.909291,0.883797,0.050987,0.041203,9.03286,"[0, 830, … 419]"
1,0.28296,0.561952,0.20464,0.91307,6.259133,"[1, 1599, … 1682]"
2,0.202197,0.700336,0.297267,0.436134,3.891523,"[2, 699, … 446]"
3,0.517782,0.076448,0.910966,0.864982,4.12844,"[3, 1908, … 474]"
4,0.208706,0.480551,0.367441,0.701084,3.39438,"[4, 19, … 136]"


In [47]:
# Get all neighbors within radius r, call them best friends
print(

df.select(
    pl.col("id"),
    pds.query_radius_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in 3d space
        index = pl.col("id"),
        r = 0.1, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).alias("best friends"),
).with_columns( # -1 to remove the point itself
    (pl.col("best friends").list.len() - 1).alias("best friends count")
).head()

)

shape: (5, 3)
┌─────┬───────────────────┬────────────────────┐
│ id  ┆ best friends      ┆ best friends count │
│ --- ┆ ---               ┆ ---                │
│ u32 ┆ list[u32]         ┆ u32                │
╞═════╪═══════════════════╪════════════════════╡
│ 0   ┆ [0, 830, … 101]   ┆ 7                  │
│ 1   ┆ [1, 1599, … 1564] ┆ 6                  │
│ 2   ┆ [2, 699, … 1564]  ┆ 10                 │
│ 3   ┆ [3, 1908, … 1666] ┆ 9                  │
│ 4   ┆ [4, 19, … 1259]   ┆ 4                  │
└─────┴───────────────────┴────────────────────┘


In [48]:
# Get ids of the k nearest neighbors and distances
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = pl.col("id"),
        k = 3, 
        dist = "l2", # actually this is squared l2
        parallel = True,
        return_dist = True
    ).alias("best_friends_w_dist")
).unnest("best_friends_w_dist").head()

id,var1,var2,var3,r,rh,idx,dist
u32,f64,f64,f64,f64,f64,list[u32],list[f64]
0,0.909291,0.883797,0.050987,0.041203,9.03286,"[0, 830, … 419]","[0.0, 0.044744, … 0.080175]"
1,0.28296,0.561952,0.20464,0.91307,6.259133,"[1, 1599, … 1682]","[0.0, 0.080255, … 0.093535]"
2,0.202197,0.700336,0.297267,0.436134,3.891523,"[2, 699, … 446]","[0.0, 0.045895, … 0.058934]"
3,0.517782,0.076448,0.910966,0.864982,4.12844,"[3, 1908, … 474]","[0.0, 0.048407, … 0.051531]"
4,0.208706,0.480551,0.367441,0.701084,3.39438,"[4, 19, … 136]","[0.0, 0.055425, … 0.088305]"


In [49]:
# Filter to only points near the given point
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5, 0.5],
        r = 0.2,
        dist = "l2" # actually this is squared l2, so this is asking for squared l2 <= 0.2
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
1,0.28296,0.561952,0.20464,0.91307,6.259133
2,0.202197,0.700336,0.297267,0.436134,3.891523
4,0.208706,0.480551,0.367441,0.701084,3.39438
7,0.412758,0.314244,0.655063,0.097273,6.469479
11,0.822353,0.80897,0.493537,0.806342,5.322478


In [50]:
# Haversine distance is available when dimension is 2
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5],
        r = 10, # in km
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
23,0.5701,0.545562,0.091339,0.19684,2.35914
55,0.495225,0.548383,0.105188,0.277119,1.723293
59,0.503864,0.522583,0.291493,0.175174,4.71879
132,0.465892,0.434237,0.980264,0.404962,5.910106
159,0.451962,0.500064,0.390375,0.778265,5.166403


In [51]:
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), 
        pt = [0.5, 0.5],
        # radius can also be an existing column in the dataframe.
        r = pl.col("rh"), 
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
59,0.503864,0.522583,0.291493,0.175174,4.71879
213,0.440951,0.528829,0.078859,0.418549,7.60724
246,0.466112,0.519506,0.412356,0.927081,8.954681
461,0.478636,0.561645,0.463417,0.961912,8.761375
516,0.527453,0.460041,0.204227,0.547982,8.697078


In [52]:
friends = df.select(
    pl.col("id").cast(pl.UInt64),
    pds.query_radius_ptwise(
        # Columns used as the coordinates in n-d space
        pl.col("var1"), pl.col("var2"), 
        index=pl.col("id"),
        r = 0.02, 
        dist = "l2",
    ).alias("friends")
).with_columns(
    pl.col("friends").list.len().alias("count")
)
friends.head()

id,friends,count
u64,list[u32],u32
0,"[0, 1863, … 1380]",4
1,[1],1
2,"[2, 1984, … 902]",4
3,"[3, 1609, … 1391]",4
4,"[4, 795]",2


# Using PDS Expressions On pl.Series, NumPy arrays, or pd.Series

In [53]:
import pandas as pd
import numpy as np
from polars_ds.compat import compat as pds2

df = pds.frame(size=100_000).select(
    pds.random(0.0, 1.0).round().alias("actual"),
    pds.random(0.0, 1.0).alias("predicted"),
    pds.random_int(0, 3).alias("0-2"),
    pds.random_int(0, 10).alias("0-9"),
    pds.random_str(min_size=1, max_size=2).alias("s1"),
    pds.random_str(min_size=1, max_size=2).alias("s2"),
)
df.head()

actual,predicted,0-2,0-9,s1,s2
f64,f64,i32,i32,str,str
0.0,0.025416,1,2,"""m""","""uL"""
1.0,0.102931,1,5,"""W""","""yL"""
1.0,0.739818,1,2,"""H""","""mF"""
1.0,0.106416,2,6,"""5""","""B"""
1.0,0.374921,1,5,"""l""","""dz"""


In [54]:
df_pd = df.to_pandas()

In [55]:
# Pandas Series
pds2.jaccard_col(df_pd["0-2"], df_pd["0-9"])

jaccard_col
f64
0.3


In [56]:
# Polars Series
pds2.query_binary_metrics(df["actual"], df["predicted"]).struct.unnest()

precision,recall,f,average_precision,roc_auc
f64,f64,f64,f64,f64
0.501429,0.502614,0.502021,0.502378,0.503615


In [57]:
# NumPy Arrays
pds2.psi(
    np.random.random(size = 1000), 
    np.random.random(size = 1000), 
    n_bins = 5, 
    return_report = True
).struct.unnest()

cnt<=,baseline_pct,actual_pct,psi_bin
f64,f64,f64,f64
0.217667,0.2,0.205,0.000123
0.415258,0.2,0.198,0.00002
0.606419,0.2,0.196,0.000081
0.813805,0.2,0.219,0.001724
inf,0.2,0.182,0.001698


In [58]:
pds2.query_cid_ce(
    np.random.random(size = 1000),
)

cid_ce
f64
12.613179


In [59]:
pds2.query_c3_stats(
    pl.Series(values=np.random.random(size = 1000)), 
    lag = 3
)

c3_stats
f64
0.116697


In [60]:
pds2.str_leven(df["s1"], df["s2"]).head() 

str_leven
u32
2
2
2
1
2
2
2
2
2
